## Rover Vision Thresholding Tool
This notebook was written to make it easy to do thresholding using GUI sliders for the different color channels.

In [64]:
%matplotlib qt
#%matplotlib qt # Choose %matplotlib qt to plot to an interactive window (note it may show up behind your browser)
# Make some of the relevant imports
import cv2 # OpenCV for perspective transform
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc # For saving images as needed
import glob  # For reading in a list of images from a folder
from matplotlib.widgets import Slider, Button
import perception

## Pre-requisite functions

In [65]:
def get_random_img():
    path = '../test_dataset/IMG/*'
    img_list = glob.glob(path)
    idx = np.random.randint(0, len(img_list)-1)
    print('Fetching random image : ', img_list[idx])
    return mpimg.imread(img_list[idx])

# Thresholding Tool

In [66]:
#test_image = get_random_img()
test_image = mpimg.imread("../new_calib/calib9.jpg")

# define constants
dst_size = 5 
bottom_offset = 6
source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
destination = np.float32([[test_image.shape[1]/2 - dst_size, test_image.shape[0] - bottom_offset],
              [test_image.shape[1]/2 + dst_size, test_image.shape[0] - bottom_offset],
              [test_image.shape[1]/2 + dst_size, test_image.shape[0] - 2*dst_size - bottom_offset], 
              [test_image.shape[1]/2 - dst_size, test_image.shape[0] - 2*dst_size - bottom_offset],
              ])
world_map_size = 200
map_scale = 10

# Function to use for the slider
slider_function = perception.rock_thresh
# replace these valuels according to the subsequent cell based on the slider function chosen
r_max = 204
r_min = 125
g_max = 185
g_min = 102
b_max = 78
b_min = 0

nav_threshed = perception.navigable_thresh(test_image)
rock_threshed = perception.rock_thresh(test_image)
obstacle_threshed = perception.obstacle_thresh(test_image)
warped_nav = perception.perspect_transform(test_image, source, destination)
warped_nav_threshed = perception.perspect_transform(nav_threshed, source, destination)
warped_obs = perception.perspect_transform(warped_nav_threshed, source, destination)

#Show images
plt.subplot(321)
plt.imshow(test_image)
plt.subplot(323)
plt.imshow(nav_threshed, cmap='gray')
plt.subplot(324)
plt.imshow(obstacle_threshed, cmap='gray')
plt.subplot(325)
plt.imshow(warped_nav, cmap='gray')
plt.subplot(326)
plt.imshow(warped_nav_threshed, cmap='gray')

plt.subplot(322)
plt.subplots_adjust(left=0.25, bottom=0.2)

v_space = 0.025
ax_rmin = plt.axes([0.25, v_space*5, 0.65, 0.02])
ax_rmax  = plt.axes([0.25, v_space*6, 0.65, 0.02])
ax_gmin = plt.axes([0.25, v_space*3, 0.65, 0.02])
ax_gmax  = plt.axes([0.25, v_space*4, 0.65, 0.02])
ax_bmin = plt.axes([0.25, v_space*1, 0.65, 0.02])
ax_bmax  = plt.axes([0.25, v_space*2, 0.65, 0.02])

s_rmin = Slider(ax_rmin, 'R min', 0, 255, valinit=r_min)
s_rmax = Slider(ax_rmax, 'R max', 0, 255, valinit=r_max)
s_gmin = Slider(ax_gmin, 'G min', 0, 255, valinit=g_min)
s_gmax = Slider(ax_gmax, 'G max', 0, 255, valinit=g_max)
s_bmin = Slider(ax_bmin, 'B min', 0, 255, valinit=b_min)
s_bmax = Slider(ax_bmax, 'B max', 0, 255, valinit=b_max)

def update(val, s=None):
    r_min = int(s_rmin.val)
    r_max = int(s_rmax.val)
    g_min = int(s_gmin.val)
    g_max = int(s_gmax.val)
    b_min = int(s_bmin.val)
    b_max = int(s_bmax.val)
    variable_image = slider_function(test_image, rgb_thresh_min=(r_min, g_min, b_min), rgb_thresh_max=(r_max,g_max,b_max))
    plt.subplot(322)
    plt.imshow(variable_image, cmap='gray')
    plt.draw()

s_rmin.on_changed(update)
s_rmax.on_changed(update)
s_gmin.on_changed(update)
s_gmax.on_changed(update)
s_bmin.on_changed(update)
s_bmax.on_changed(update)

update(val = None)

## Common Parameters 

### obstacle : 
```
r_max = 118
r_min = 0
g_max = 103
g_min = 0
b_max = 120
b_min = 0
```

### navigable
```
r_max = 255
r_min = 118
g_max = 255
g_min = 93
b_max = 255
b_min = 89
```

### rock
```
r_max = 204
r_min = 125
g_max = 185
g_min = 102
b_max = 78
b_min = 0
```

### Tyre marks
```
r_max = 207
r_min = 130
g_max = 182
g_min = 107
b_max = 159
b_min = 83
```